In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

In [2]:
df=pd.read_csv('toronto_geocoded_postcodes.csv')
df.head()

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.648690,-79.385440
1,1,M4A,North York,Victoria Village,43.648690,-79.385440
2,2,M5A,Downtown Toronto,Harbourfront,43.752440,-79.329271
3,3,M5A,Downtown Toronto,Regent Park,43.730421,-79.313320
4,4,M6A,North York,Lawrence Heights,43.655120,-79.362640


In [7]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [8]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.648690,-79.385440
1,M4A,North York,Victoria Village,43.648690,-79.385440
2,M5A,Downtown Toronto,Harbourfront,43.752440,-79.329271
3,M5A,Downtown Toronto,Regent Park,43.730421,-79.313320
4,M6A,North York,Lawrence Heights,43.655120,-79.362640


# Finding Toronto Long. and Lat.

In [9]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [11]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Getting Data From Foursquare

In [12]:
CLIENT_ID = 'LAQYXVWOC5L2KKIRAZE53NGRHARECARN4BCCML0UEH552NM3' # your Foursquare ID
CLIENT_SECRET = 'TMIJ1QXQFNVVTFU40P1ATPEXVUNPVCDXYW1NMKPU4JD4LQVW' # your Foursquare Secret
VERSION = '20180604'

In [13]:
def getBuiltUrl(neigh_lat,neigh_long,radius=1400):
    # type your answer here
    LIMIT=100
    #radius=1000
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_long, 
    radius, 
    LIMIT)
    return url
    #url

In [14]:
neigh_post, neigh_borough, neigh_name, neigh_lat, neigh_long = df.iloc[0]
print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_lat, 
                                                               neigh_long))
results = requests.get(getBuiltUrl(neigh_lat,neigh_long)).json()
results

Latitude and longitude values of Parkwoods are 43.648690000000045, -79.38543999999995.


{'meta': {'code': 200, 'requestId': '5d2c2d19bf7dde002cd60fde'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Entertainment District',
  'headerFullLocation': 'Entertainment District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 239,
  'suggestedBounds': {'ne': {'lat': 43.661290012600055,
    'lng': -79.36805921241036},
   'sw': {'lat': 43.636089987400034, 'lng': -79.40282078758953}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e31b74252b131dcebb08743',
       'name': 'Shangri-La Toronto',
       'location': {'address': '188 University Ave.',
        'crossStreet': 'at Adelaide St. W',
        'lat': 43.64912919417502,
        'lng': -79.386556

## Cleaning Data

In [15]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Shangri-La Toronto,Hotel,43.649129,-79.386557
1,Soho House Toronto,Speakeasy,43.648734,-79.386541
2,Pai,Thai Restaurant,43.647923,-79.388579
3,Byblos Toronto,Mediterranean Restaurant,43.647615,-79.388381
4,The Keg Steakhouse & Bar,Steakhouse,43.649937,-79.384196


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


# Nearby Venues

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        LIMIT=100
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'], 
                                   latitudes=df['Latitude'], 
                                   longitudes=df['Longitude'], 
                                   radius=4000)

Parkwoods
Victoria Village
Harbourfront
Regent Park
Lawrence Heights
Lawrence Manor
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design Exchange
Toron

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(20831, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.64869,-79.38544,Pai,43.647923,-79.388579,Thai Restaurant
1,Parkwoods,43.64869,-79.38544,Byblos Toronto,43.647615,-79.388381,Mediterranean Restaurant
2,Parkwoods,43.64869,-79.38544,Shangri-La Toronto,43.649129,-79.386557,Hotel
3,Parkwoods,43.64869,-79.38544,Soho House Toronto,43.648734,-79.386541,Speakeasy
4,Parkwoods,43.64869,-79.38544,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall


In [23]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,100,100,100,100,100,100
Agincourt North,100,100,100,100,100,100
Albion Gardens,100,100,100,100,100,100
Alderwood,100,100,100,100,100,100
Bathurst Manor,76,76,76,76,76,76
Bathurst Quay,100,100,100,100,100,100
Bayview Village,100,100,100,100,100,100
Beaumond Heights,100,100,100,100,100,100


In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 249 uniques categories.


# One Hot Encoding on the venues

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo Exhibit,Afghan Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Liquor Store,Lounge,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [27]:
toronto_onehot.shape

(20831, 249)

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Afghan Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Liquor Store,Lounge,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Adelaide,0.000000,0.000000,0.00,0.010000,0.010,0.00,0.02,0.00,0.000,0.010000,0.000,0.000,0.00,0.00,0.010000,0.000000,0.00,0.000000,0.010,0.00,0.010,0.00,0.000000,0.010,0.000000,0.00,0.00,0.010000,0.010000,0.000,0.000,0.00,0.000000,0.01,0.00,0.00,0.000,0.00,0.000000,0.010,0.000000,0.000000,0.030000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.010000,0.01,0.000,0.010000,0.000,0.110000,0.000,0.010,0.010,0.02,0.00,0.00,0.010000,0.010,0.0

In [29]:
toronto_grouped.shape

(208, 249)

## Top 5 venues

In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
         venue  freq
0  Coffee Shop  0.11
1      Theater  0.04
2        Hotel  0.04
3          Gym  0.03
4  Yoga Studio  0.03


----Agincourt----
                  venue  freq
0           Coffee Shop  0.07
1                  Park  0.06
2                   Pub  0.04
3  Fast Food Restaurant  0.04
4                 Beach  0.03


----Agincourt North----
          venue  freq
0          Café  0.05
1          Park  0.05
2   Coffee Shop  0.05
3        Bakery  0.04
4  Burger Joint  0.04


----Albion Gardens----
         venue  freq
0         Café  0.11
1          Bar  0.09
2         Park  0.05
3       Bakery  0.05
4  Coffee Shop  0.05


----Alderwood----
                           venue  freq
0                           Café  0.09
1                 Sandwich Place  0.06
2                    Pizza Place  0.04
3                           Park  0.04
4  Vegetarian / Vegan Restaurant  0.04


----Bathurst Manor----
            venue  freq
0     Coffee Shop  0.13
1  Sandwich Place  0.

                 venue  freq
0         Burger Joint  0.05
1  Japanese Restaurant  0.05
2        Grocery Store  0.04
3               Bakery  0.04
4                 Café  0.04


----Downsview West----
         venue  freq
0         Park  0.07
1  Yoga Studio  0.05
2        Hotel  0.05
3     Beer Bar  0.04
4       Bakery  0.04


----Dufferin----
                venue  freq
0         Coffee Shop  0.07
1         Yoga Studio  0.04
2               Hotel  0.04
3                Park  0.03
4  Italian Restaurant  0.03


----East Birchmount Park----
            venue  freq
0     Coffee Shop  0.07
1  Clothing Store  0.07
2      Restaurant  0.04
3   Grocery Store  0.03
4            Café  0.03


----East Toronto----
                venue  freq
0         Coffee Shop  0.07
1         Yoga Studio  0.04
2               Hotel  0.04
3                Park  0.03
4  Italian Restaurant  0.03


----Emery----
                venue  freq
0         Coffee Shop  0.07
1         Yoga Studio  0.04
2               Hotel 

                venue  freq
0         Coffee Shop  0.07
1         Yoga Studio  0.04
2               Hotel  0.04
3                Park  0.03
4  Italian Restaurant  0.03


----King's Mill Park----
            venue  freq
0            Café  0.07
1     Coffee Shop  0.05
2  Sandwich Place  0.05
3          Bakery  0.05
4     Yoga Studio  0.04


----Kingsview Village----
                venue  freq
0         Coffee Shop  0.07
1         Yoga Studio  0.04
2               Hotel  0.04
3                Park  0.03
4  Italian Restaurant  0.03


----Kingsway Park South East----
            venue  freq
0            Café  0.07
1  Sandwich Place  0.05
2     Coffee Shop  0.05
3     Pizza Place  0.04
4     Yoga Studio  0.04


----Kingsway Park South West----
                  venue  freq
0    Chinese Restaurant  0.15
1       Bubble Tea Shop  0.04
2           Coffee Shop  0.04
3  Caribbean Restaurant  0.04
4                Bakery  0.04


----L'Amoreaux East----
                venue  freq
0         Coffee 

                venue  freq
0         Coffee Shop  0.07
1         Yoga Studio  0.04
2               Hotel  0.04
3                Park  0.03
4  Italian Restaurant  0.03


----Port Union----
           venue  freq
0    Coffee Shop  0.08
1   Burger Joint  0.07
2   Liquor Store  0.05
3  Grocery Store  0.04
4         Bakery  0.03


----Princess Gardens----
                 venue  freq
0          Coffee Shop  0.09
1                Hotel  0.04
2                 Café  0.04
3          Pizza Place  0.04
4  Japanese Restaurant  0.04


----Railway Lands----
                venue  freq
0         Coffee Shop  0.08
1                Café  0.07
2  Italian Restaurant  0.06
3              Bakery  0.06
4                Park  0.04


----Rathnelly----
                venue  freq
0                Café  0.12
1         Coffee Shop  0.08
2                Park  0.06
3  Italian Restaurant  0.06
4                 Bar  0.06


----Regent Park----
                       venue  freq
0  Middle Eastern Restaurant  0.09


                  venue  freq
0           Coffee Shop  0.12
1                  Park  0.08
2  Fast Food Restaurant  0.07
3        Sandwich Place  0.07
4                  Bank  0.05


----The Kingsway----
                  venue  freq
0    Chinese Restaurant  0.09
1           Coffee Shop  0.06
2  Caribbean Restaurant  0.06
3                Bakery  0.04
4        Breakfast Spot  0.03


----The Queensway East----
         venue  freq
0  Coffee Shop  0.08
1  Yoga Studio  0.04
2        Hotel  0.04
3          Gym  0.03
4  Pizza Place  0.03


----The Queensway West----
                  venue  freq
0    Chinese Restaurant  0.15
1       Bubble Tea Shop  0.04
2           Coffee Shop  0.04
3  Caribbean Restaurant  0.04
4                Bakery  0.04


----Thistletown----
                  venue  freq
0           Coffee Shop  0.07
1    Chinese Restaurant  0.07
2  Caribbean Restaurant  0.05
3     Indian Restaurant  0.04
4          Noodle House  0.03


----Thorncliffe Park----
         venue  freq
0  

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Hotel,Theater,Farmers Market,Park,Café,Restaurant,Yoga Studio,Japanese Restaurant,Gym
1,Agincourt,Coffee Shop,Park,Fast Food Restaurant,Pub,Bank,Sandwich Place,Burger Joint,Breakfast Spot,Beach,Vegetarian / Vegan Restaurant
2,Agincourt North,Coffee Shop,Café,Park,Burger Joint,Sushi Restaurant,Grocery Store,Bakery,Bagel Shop,Steakhouse,Supermarket
3,Albion Gardens,Café,Bar,Coffee Shop,Bakery,Park,Restaurant,Tea Room,Italian Restaurant,Eastern European Restaurant,Cocktail Bar
4,Alderwood,Café,Sandwich Place,Park,Pizza Place,Vegetarian / Vegan Restaurant,Cocktail Bar,Bar,Coffee Shop,Restaurant,Yoga Studio


# Performing K-means Clustering

In [34]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 3, 3, 0, 2, 1, 1, 1])

In [44]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.648690,-79.385440,1,Coffee Shop,Yoga Studio,Hotel,Italian Restaurant,Pizza Place,Park,Steakhouse,Sandwich Place,Gym,Café
1,M4A,North York,Victoria Village,43.648690,-79.385440,1,Coffee Shop,Yoga Studio,Hotel,Italian Restaurant,Pizza Place,Park,Steakhouse,Sandwich Place,Gym,Café
2,M5A,Downtown Toronto,Harbourfront,43.752440,-79.329271,2,Middle Eastern Restaurant,Coffee Shop,Supermarket,Japanese Restaurant,Park,Ice Cream Shop,Café,Seafood Restaurant,Chinese Restaurant,Mediterranean Restaurant
3,M5A,Downtown Toronto,Regent Park,43.730421,-79.313320,2,Middle Eastern Restaurant,Supermarket,Burger Joint,Japanese Restaurant,Gym / Fitness Center,Park,Coffee Shop,Indian Restaurant,Liquor Store,Burrito Place
4,M6A,North York,Lawrence Heights,43.655120,-79.362640,1,Coffee Shop,Japanese Restaurant,Park,Café,Hotel,Farmers Market,Steakhouse,Restaurant,Diner,Gastropub


# Visualizing Clusters

In [45]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 1

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,East York,0,Zoo Exhibit,Coffee Shop,Fast Food Restaurant,Pharmacy,Pizza Place,Sandwich Place,Park,Burger Joint,Discount Store,Zoo
12,Downtown Toronto,0,Zoo Exhibit,Coffee Shop,Fast Food Restaurant,Pharmacy,Pizza Place,Sandwich Place,Park,Burger Joint,Discount Store,Zoo
27,York,0,Coffee Shop,Fast Food Restaurant,Pizza Place,Park,Sandwich Place,Pharmacy,Breakfast Spot,Supermarket,Fried Chicken Joint,Burger Joint
28,Etobicoke,0,Coffee Shop,Fast Food Restaurant,Pizza Place,Park,Sandwich Place,Pharmacy,Breakfast Spot,Supermarket,Fried Chicken Joint,Burger Joint
29,Etobicoke,0,Coffee Shop,Fast Food Restaurant,Pizza Place,Park,Sandwich Place,Pharmacy,Breakfast Spot,Supermarket,Fried Chicken Joint,Burger Joint
42,Scarborough,0,Coffee Shop,Park,Sandwich Place,Fast Food Restaurant,Pharmacy,Pizza Place,Supermarket,Grocery Store,Beer Store,Burger Joint
43,North York,0,Coffee Shop,Park,Sandwich Place,Fast Food Restaurant,Pharmacy,Pizza Place,Supermarket,Grocery Store,Beer Store,Burger Joint
44,North York,0,Coffee Shop,Park,Sandwich Place,Fast Food Restaurant,Pharmacy,Pizza Place,Supermarket,Grocery Store,Beer Store,Burger Joint
53,Scarborough,0,Coffee Shop,Indian Restaurant,Clothing Store,Pizza Place,Caribbean Restaurant,Pharmacy,Park,Fast Food Restaurant,Bank,Gym
76,West Toronto,0,Coffee Shop,Pharmacy,Sandwich Place,Fast Food Restaurant,Chinese Restaurant,Indian Restaurant,Grocery Store,Bank,Pizza Place,Burger Joint


# Cluster 2

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Coffee Shop,Yoga Studio,Hotel,Italian Restaurant,Pizza Place,Park,Steakhouse,Sandwich Place,Gym,Café
1,North York,1,Coffee Shop,Yoga Studio,Hotel,Italian Restaurant,Pizza Place,Park,Steakhouse,Sandwich Place,Gym,Café
4,North York,1,Coffee Shop,Japanese Restaurant,Park,Café,Hotel,Farmers Market,Steakhouse,Restaurant,Diner,Gastropub
5,North York,1,Coffee Shop,Japanese Restaurant,Park,Café,Hotel,Farmers Market,Steakhouse,Restaurant,Diner,Gastropub
8,Scarborough,1,Café,Coffee Shop,Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Concert Hall,Sandwich Place,Japanese Restaurant,Yoga Studio,Park
9,North York,1,Coffee Shop,Yoga Studio,Hotel,Italian Restaurant,Pizza Place,Park,Steakhouse,Sandwich Place,Gym,Café
13,Downtown Toronto,1,Coffee Shop,Yoga Studio,Hotel,Italian Restaurant,Pizza Place,Park,Steakhouse,Sandwich Place,Gym,Café
17,Etobicoke,1,Coffee Shop,Pizza Place,Café,Japanese Restaurant,Hotel,Farmers Market,Steakhouse,Concert Hall,Restaurant,Park
18,Etobicoke,1,Coffee Shop,Pizza Place,Café,Japanese Restaurant,Hotel,Farmers Market,Steakhouse,Concert Hall,Restaurant,Park
20,Scarborough,1,Coffee Shop,Yoga Studio,Hotel,Italian Restaurant,Pizza Place,Park,Steakhouse,Sandwich Place,Gym,Café


# Cluster 3

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,Middle Eastern Restaurant,Coffee Shop,Supermarket,Japanese Restaurant,Park,Ice Cream Shop,Café,Seafood Restaurant,Chinese Restaurant,Mediterranean Restaurant
3,Downtown Toronto,2,Middle Eastern Restaurant,Supermarket,Burger Joint,Japanese Restaurant,Gym / Fitness Center,Park,Coffee Shop,Indian Restaurant,Liquor Store,Burrito Place
6,Etobicoke,2,Clothing Store,Coffee Shop,Furniture / Home Store,Bakery,Dessert Shop,Café,Latin American Restaurant,Sandwich Place,Liquor Store,Restaurant
7,Scarborough,2,Clothing Store,Coffee Shop,Furniture / Home Store,Bakery,Dessert Shop,Café,Latin American Restaurant,Sandwich Place,Liquor Store,Restaurant
10,East York,2,Coffee Shop,Bakery,Liquor Store,Burger Joint,Pharmacy,Bank,Café,Pub,Italian Restaurant,Grocery Store
14,North York,2,Park,Japanese Restaurant,Middle Eastern Restaurant,Supermarket,Bakery,Gym / Fitness Center,Furniture / Home Store,Coffee Shop,Ice Cream Shop,Burger Joint
15,Etobicoke,2,Gastropub,Bakery,Middle Eastern Restaurant,Coffee Shop,Café,Brewery,Gym / Fitness Center,Turkish Restaurant,Asian Restaurant,Indian Restaurant
16,Etobicoke,2,Gastropub,Bakery,Middle Eastern Restaurant,Coffee Shop,Café,Brewery,Gym / Fitness Center,Turkish Restaurant,Asian Restaurant,Indian Restaurant
19,Etobicoke,2,Clothing Store,Coffee Shop,Bakery,Café,Italian Restaurant,Furniture / Home Store,BBQ Joint,Breakfast Spot,Jewelry Store,Ice Cream Shop
22,Scarborough,2,Coffee Shop,Burger Joint,Liquor Store,Grocery Store,Breakfast Spot,Furniture / Home Store,Sporting Goods Shop,Seafood Restaurant,Sandwich Place,Bakery


# Cluster 4

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Scarborough,3,Park,Brewery,Coffee Shop,Beach,Gastropub,Breakfast Spot,Café,Indian Restaurant,Ice Cream Shop,Pizza Place
35,East Toronto,3,Coffee Shop,Café,Italian Restaurant,Bakery,Ice Cream Shop,Park,Middle Eastern Restaurant,Spa,Indian Restaurant,Breakfast Spot
47,East York,3,Park,Coffee Shop,Beach,Café,Brewery,Bakery,Gastropub,Ice Cream Shop,Breakfast Spot,Pub
49,Downtown Toronto,3,Café,Italian Restaurant,Coffee Shop,Bar,Ice Cream Shop,Bakery,Indian Restaurant,Brewery,Gastropub,BBQ Joint
56,North York,3,Park,Coffee Shop,Italian Restaurant,Grocery Store,Café,Bakery,Pizza Place,Restaurant,Indian Restaurant,Liquor Store
58,North York,3,Café,Bar,Italian Restaurant,Park,Vegetarian / Vegan Restaurant,Cocktail Bar,Indian Restaurant,Sandwich Place,Middle Eastern Restaurant,Asian Restaurant
67,Scarborough,3,Café,Park,Greek Restaurant,Grocery Store,Brewery,Gastropub,Bakery,Ice Cream Shop,Farmers Market,Furniture / Home Store
71,East Toronto,3,Café,Bar,Cocktail Bar,Coffee Shop,Restaurant,Brewery,Park,Italian Restaurant,Asian Restaurant,Bakery
72,East Toronto,3,Café,Bar,Cocktail Bar,Coffee Shop,Restaurant,Brewery,Park,Italian Restaurant,Asian Restaurant,Bakery
82,North York,3,Café,Greek Restaurant,Park,Brewery,Coffee Shop,Bakery,Indian Restaurant,Gastropub,Farmers Market,Pizza Place


# Cluster 5

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Downtown Toronto,4,Coffee Shop,Indian Restaurant,Chinese Restaurant,Gym,Bakery,Clothing Store,Pharmacy,Grocery Store,Bookstore,Caribbean Restaurant
63,West Toronto,4,Chinese Restaurant,Bakery,Japanese Restaurant,Coffee Shop,Caribbean Restaurant,Asian Restaurant,Sporting Goods Shop,Supermarket,Electronics Store,Hotel
77,West Toronto,4,Bakery,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Middle Eastern Restaurant,Pharmacy,Supermarket,Seafood Restaurant,Steakhouse,Sushi Restaurant
78,Scarborough,4,Bakery,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Middle Eastern Restaurant,Pharmacy,Supermarket,Seafood Restaurant,Steakhouse,Sushi Restaurant
79,Scarborough,4,Bakery,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Middle Eastern Restaurant,Pharmacy,Supermarket,Seafood Restaurant,Steakhouse,Sushi Restaurant
151,Downtown Toronto,4,Coffee Shop,Chinese Restaurant,Indian Restaurant,Clothing Store,Middle Eastern Restaurant,Grocery Store,Breakfast Spot,Supermarket,Pizza Place,Caribbean Restaurant
152,Downtown Toronto,4,Coffee Shop,Chinese Restaurant,Indian Restaurant,Middle Eastern Restaurant,Grocery Store,Clothing Store,Caribbean Restaurant,Pizza Place,Supermarket,Gym
153,Downtown Toronto,4,Coffee Shop,Chinese Restaurant,Indian Restaurant,Middle Eastern Restaurant,Grocery Store,Clothing Store,Caribbean Restaurant,Pizza Place,Supermarket,Gym
180,Etobicoke,4,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Indian Restaurant,Bakery,Clothing Store,Noodle House,Bubble Tea Shop,Tea Room,Mexican Restaurant
191,Downtown Toronto,4,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Bakery,Noodle House,Breakfast Spot,Fast Food Restaurant,Vietnamese Restaurant,Supermarket,Falafel Restaurant
